In [1]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Using TensorFlow backend.


In [2]:
x_train.shape

(50000, 32, 32, 3)

In [3]:
x_train[0:5
       ]

array([[[[ 59,  62,  63],
         [ 43,  46,  45],
         [ 50,  48,  43],
         ..., 
         [158, 132, 108],
         [152, 125, 102],
         [148, 124, 103]],

        [[ 16,  20,  20],
         [  0,   0,   0],
         [ 18,   8,   0],
         ..., 
         [123,  88,  55],
         [119,  83,  50],
         [122,  87,  57]],

        [[ 25,  24,  21],
         [ 16,   7,   0],
         [ 49,  27,   8],
         ..., 
         [118,  84,  50],
         [120,  84,  50],
         [109,  73,  42]],

        ..., 
        [[208, 170,  96],
         [201, 153,  34],
         [198, 161,  26],
         ..., 
         [160, 133,  70],
         [ 56,  31,   7],
         [ 53,  34,  20]],

        [[180, 139,  96],
         [173, 123,  42],
         [186, 144,  30],
         ..., 
         [184, 148,  94],
         [ 97,  62,  34],
         [ 83,  53,  34]],

        [[177, 144, 116],
         [168, 129,  94],
         [179, 142,  87],
         ..., 
         [216, 184, 140],
  

In [4]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.
It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))



x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])



In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        fill_mode='nearest',  # set mode for filling points outside the input boundaries
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        rescale=None,  # set rescaling factor (applied before any other transformation)
        preprocessing_function=None,  # set function that will be applied on each input
        data_format=None,  # image data format, either "channels_first" or "channels_last"
        validation_split=0.0)  # fraction of images reserved for validation (strictly between 0 and 1)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])

Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 248s 159ms/step - loss: 1.9140 - acc: 0.2962 - val_loss: 1.5908 - val_acc: 0.4357
Epoch 2/100
1563/1563 [==============================] - 248s 159ms/step - loss: 1.6124 - acc: 0.4106 - val_loss: 1.5171 - val_acc: 0.4606
Epoch 3/100
1563/1563 [==============================] - 248s 159ms/step - loss: 1.4829 - acc: 0.4625 - val_loss: 1.2977 - val_acc: 0.5290
Epoch 4/100
1563/1563 [==============================] - 248s 158ms/step - loss: 1.3942 - acc: 0.4999 - val_loss: 1.2804 - val_acc: 0.5470
Epoch 5/100
1563/1563 [==============================] - 247s 158ms/step - loss: 1.3210 - acc: 0.5284 - val_loss: 1.1492 - val_acc: 0.5953
Epoch 6/100
1563/1563 [==============================] - 248s 159ms/step - loss: 1.2588 - acc: 0.5499 - val_loss: 1.1131 - val_acc: 0.6056
Epoch 7/100
1563/1563 [==============================] - 248s 159ms/step - loss: 1.2079 - acc: 0.5717 - val_loss: 1.0783 - val_acc:

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1563/1563 [==============================] - 948s 607ms/step - loss: 0.8699 - acc: 0.7038 - val_loss: 0.7907 - val_acc: 0.7343
Epoch 29/100
1563/1563 [==============================] - 912s 583ms/step - loss: 0.8610 - acc: 0.7047 - val_loss: 0.7712 - val_acc: 0.7403
Epoch 30/100
 554/1563 [=========>....................] - ETA: 9:15 - loss: 0.8548 - acc: 0.7071

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1563/1563 [==============================] - 924s 591ms/step - loss: 0.8566 - acc: 0.7060 - val_loss: 0.7202 - val_acc: 0.7554
Epoch 32/100
1563/1563 [==============================] - 963s 616ms/step - loss: 0.8510 - acc: 0.7102 - val_loss: 0.7674 - val_acc: 0.7417
Epoch 33/100
 469/1563 [========>.....................] - ETA: 9:47 - loss: 0.8492 - acc: 0.7109

...

1563/1563 [==============================] - 954s 610ms/step - loss: 0.8098 - acc: 0.7321 - val_loss: 0.7271 - val_acc: 0.7719
Epoch 88/100
1563/1563 [==============================] - 928s 593ms/step - loss: 0.8115 - acc: 0.7330 - val_loss: 0.7556 - val_acc: 0.7643
Epoch 89/100
 552/1563 [=========>....................] - ETA: 10:01 - loss: 0.7989 - acc: 0.7387

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1563/1563 [==============================] - 1061s 679ms/step - loss: 0.8141 - acc: 0.7318 - val_loss: 0.6960 - val_acc: 0.7744
Epoch 91/100
1563/1563 [==============================] - 915s 586ms/step - loss: 0.8124 - acc: 0.7302 - val_loss: 0.6820 - val_acc: 0.7794
Epoch 92/100
1563/1563 [==============================] - 962s 615ms/step - loss: 0.8192 - acc: 0.7299 - val_loss: 0.7047 - val_acc: 0.7654
Epoch 93/100
1563/1563 [==============================] - 971s 621ms/step - loss: 0.8231 - acc: 0.7290 - val_loss: 0.7364 - val_acc: 0.7570
Epoch 94/100
1563/1563 [==============================] - 966s 618ms/step - loss: 0.8166 - acc: 0.7321 - val_loss: 0.7519 - val_acc: 0.7539
Epoch 95/100
1563/1563 [==============================] - 920s 589ms/step - loss: 0.8231 - acc: 0.7293 - val_loss: 0.7563 - val_acc: 0.7466
Epoch 96/100
1563/1563 [==============================] - 948s 606ms/step - loss: 0.8244 - acc: 0.7308 - val_loss: 0.7266 - val_acc: 0.7577
Epoch 97/100
1563/1563 [========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1563/1563 [==============================] - 906s 580ms/step - loss: 0.8361 - acc: 0.7243 - val_loss: 0.8039 - val_acc: 0.7379
Saved trained model at /home/jovyan/cifar-10/saved_models/keras_cifar10_trained_model.h5 
10000/10000 [==============================] - 40s 4ms/step
Test loss: 0.8039314188
